In [1]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

import requests
import csv

import warnings

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

# Parameter settings
alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [8]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Information Technology']['Ticker'].tolist()

In [5]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBY, BKNG, CZR, KMX, C...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, APO, ACGL, AJG,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, TECH, BII...","[MMM, AOS, ALLE, AME, ADP, AXON, BA, BR, BLDR,...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CF, CTVA, DOW, DD,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [9]:
pivot_sector_ticker_df['Information Technology'][0]

['ACN',
 'ADBE',
 'AMD',
 'AKAM',
 'APH',
 'ADI',
 'ANSS',
 'AAPL',
 'AMAT',
 'ANET',
 'ADSK',
 'AVGO',
 'CDNS',
 'CDW',
 'CSCO',
 'CTSH',
 'GLW',
 'CRWD',
 'DELL',
 'ENPH',
 'EPAM',
 'FFIV',
 'FICO',
 'FSLR',
 'FTNT',
 'IT',
 'GEN',
 'GDDY',
 'HPE',
 'HPQ',
 'IBM',
 'INTC',
 'INTU',
 'JBL',
 'JNPR',
 'KEYS',
 'KLAC',
 'LRCX',
 'MCHP',
 'MU',
 'MSFT',
 'MPWR',
 'MSI',
 'NTAP',
 'NVDA',
 'NXPI',
 'ON',
 'ORCL',
 'PLTR',
 'PANW',
 'PTC',
 'QCOM',
 'ROP',
 'CRM',
 'STX',
 'NOW',
 'SWKS',
 'SMCI',
 'SNPS',
 'TEL',
 'TDY',
 'TER',
 'TXN',
 'TRMB',
 'TYL',
 'VRSN',
 'WDC',
 'WDAY',
 'ZBRA']

In [7]:
sector_overview_dict = {}

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

# create empty dataframe with columns
sector_overview_df = pd.DataFrame(columns = data.keys())

for col in pivot_sector_ticker_df:

    # create empty dataframe with columns
    sector_overview_df = pd.DataFrame(columns = data.keys())



    for symbol in pivot_sector_ticker_df[col][0]:
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
        r = requests.get(url)
        data = r.json()
        

        temp_data_df = pd.DataFrame(data.items()).transpose()

        # Check if DataFrame is not empty and has more than one row        
        if not temp_data_df.empty and len(temp_data_df) > 1:
            temp_data_df.columns = temp_data_df.iloc[0]
            temp_data_df = temp_data_df[1::]
            
            sector_overview_df = pd.concat([sector_overview_df, temp_data_df], ignore_index=True)

        else:
            print(f"{symbol} DataFrame is empty or does not have enough rows to set the first row as header.")



    # append each sector's dataframe to the dictionary
    sector_overview_dict[col] = sector_overview_df

    # # output the sector dataframe to excel
    # sector_overview_df.to_excel(
    # f'sp500_{col}.xlsx'
    # ,index=False
    # )

BF.B DataFrame is empty or does not have enough rows to set the first row as header.
BRK.B DataFrame is empty or does not have enough rows to set the first row as header.


In [8]:
print(f'This is {pivot_sector_ticker_df.columns[0]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[0]]

This is Communication Services dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,GOOGL,Common Stock,Alphabet Inc Class A,Alphabet Inc. is an American multinational con...,1652044,NASDAQ,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...","1600 AMPHITHEATRE PARKWAY, MOUNTAIN VIEW, CA, US",December,2024-03-31,2170051101000,109723001000,26.8,1.644,23.65,None,None,6.53,25.37,0.259,0.325,0.156,0.298,318145987000,156633000000,6.53,0.609,0.154,191.49,14,29,9,0,0,26.8,23.2,6.41,6.92,6.14,19.27,1.019,178.77,115.35,160.61,142.58,5874000000,2024-06-17,2024-06-10
1,GOOG,Common Stock,Alphabet Inc Class C,Alphabet Inc. is an American multinational con...,1652044,NASDAQ,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...","1600 AMPHITHEATRE PARKWAY, MOUNTAIN VIEW, CA, US",December,2024-03-31,2170058047000,109723001000,27.04,1.659,23.65,None,None,6.52,25.37,0.259,0.325,0.156,0.298,318145987000,156633000000,6.52,0.609,0.154,170.44,6,7,3,0,0,27.04,23.42,6.41,6.99,6.14,19.27,1.019,179.95,115.83,162.07,143.85,5617000000,2024-06-17,2024-06-10
2,T,Common Stock,AT&T Inc,AT&T Inc. is an American multinational conglom...,732717,NYSE,USD,USA,TECHNOLOGY,TELEPHONE COMMUNICATIONS (NO RADIOTELEPHONE),"208 S. AKARD ST, ATTN : JAMES LACY, DALLAS, TX...",December,2024-03-31,125477970000,42146001000,9.41,1.272,14.58,1.11,0.0634,1.86,17.02,0.111,0.22,0.0406,0.13,122316997000,69893000000,1.86,-0.174,-0.004,18.1,5,8,11,3,0,9.41,7.37,0.956,1.133,2.166,5.85,0.7,17.87,12.76,17.03,16.21,7170170000,2024-05-01,2024-04-09
3,CHTR,Common Stock,Charter Communications Inc,"Charter Communications, Inc., is an American t...",1091667,NASDAQ,USD,USA,TECHNOLOGY,CABLE & OTHER PAY TELEVISION SERVICES,"400 ATLANTIC STREET, STAMFORD, CT, US",December,2024-03-31,39098384000,21126001000,8.79,0.277,82.44,None,None,30.89,370.45,0.085,0.226,0.0531,0.376,54633001000,24485000000,30.89,0.135,0.002,334.76,3,5,16,2,0,8.79,7.61,0.689,3.393,2.466,6.49,0.973,458.3,236.08,272.76,357.8,144030000,None,None
4,CMCSA,Common Stock,Comcast Corp,Comcast Corporation is an American telecommuni...,1166691,NASDAQ,USD,USA,TECHNOLOGY,CABLE & OTHER PAY TELEVISION SERVICES,"ONE COMCAST CENTER, PHILADELPHIA, PA, US",December,2024-03-31,151216701000,37586002000,10.22,0.642,20.98,1.18,0.0322,3.77,30.04,0.126,0.193,0.0561,0.181,121939001000,83214000000,3.77,0.066,0.012,48.04,5,12,14,0,0,10.22,9.37,1.286,1.891,2.034,6.36,0.99,46.76,36.43,40.2,42.77,3914180000,2024-07-24,2024-07-03
5,EA,Common Stock,Electronic Arts Inc,Electronic Arts Inc. (EA) is an American video...,712515,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"209 REDWOOD SHORES PARKWAY, REDWOOD CITY, CA, US",March,2024-03-31,35977146000,1982000000,28.8,1.266,28.24,0.76,0.0056,4.69,28.01,0.168,0.204,0.0767,0.172,7562000000,5754000000,4.69,0.466,-0.051,149.7,9,8,13,0,0,28.8,16.92,4.465,4.539,4.315,16.54,0.777,144.34,117.15,129.56,130.96,266379000,2024-06-19,2024-05-29
6,FOXA,Common Stock,Fox Corp Class A,Fox Corporation is an American mass media comp...,1754301,NASDAQ,USD,USA,TECHNOLOGY,TELEVISION BROADCASTING STATIONS,"1211 AVENUE OF THE AMERICAS, NEW YORK, NY, US",June,2024-03-31,15102587000,2812000000,10.6,1.009,22.54,0.52,0.0155,3.17,28.54,0.112,0.229,0.0681,0.15,13920000000,5224000000,3.17,-0.608,-0.156,37.6,2,6,14,2,0,10.6,8.07,0.942,1.394,1.306,9.84,0.781,34.81,28.15,31.55,30.97,231150000,2024-03-

In [9]:
print(f'This is {pivot_sector_ticker_df.columns[1]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[1]]

This is Consumer Discretionary dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,ABNB,Common Stock,Airbnb Inc,"Airbnb, Inc. is an American company that opera...",1559720,NASDAQ,USD,USA,TRADE & SERVICES,SERVICES-TO DWELLINGS & OTHER BUILDINGS,"888 BRANNAN ST., SAN FRANCISCO, CA, US",December,2024-03-31,91715920000,1658000000,19.34,10.04,12.41,None,None,7.47,16.05,0.482,0.0472,0.0456,0.749,10241000000,6900000000,7.47,1.301,0.178,152.41,4,7,27,5,1,19.34,35.97,10.25,12.45,9.47,42.13,1.246,170.1,103.74,157.56,142.22,441500000,None,None
1,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,"410 TERRY AVENUE NORTH, SEATTLE, WA, US",December,2024-03-31,1880992907000,96609001000,50.49,2.32,20.83,None,None,3.58,57.13,0.0638,0.107,0.0595,0.203,590739997000,225152000000,3.58,2.167,0.125,209.47,19,39,2,0,0,50.49,44.05,3.369,9.59,3.454,22.21,1.155,191.7,118.35,181.92,156.04,10406600000,None,None
2,APTV,Common Stock,Aptiv PLC,Aptiv plc is an auto parts company headquarter...,1521332,NYSE,USD,USA,MANUFACTURING,MOTOR VEHICLE PARTS & ACCESSORIES,"5 HANOVER QUAY, GRAND CANAL DOCK, DUBLIN, IE",December,2024-03-31,22458720000,2871000000,7.76,0.675,40.88,None,None,10.64,72.43,0.149,0.0988,0.0538,0.292,20134001000,2646000000,10.64,0.466,0.017,101.82,8,12,4,2,0,7.76,13.26,1.009,1.752,1.266,10.01,1.909,113.6,68.84,77.29,85.03,272062000,2020-02-19,2020-02-04
3,AZO,Common Stock,AutoZone Inc,"AutoZone, Inc. is an American retailer of afte...",866787,NYSE,USD,USA,TRADE & SERVICES,RETAIL-AUTO & HOME SUPPLY STORES,"123 SOUTH FRONT ST, MEMPHIS, TN, US",August,2024-02-29,47881474000,4192627000,19.72,1.479,-282.21,None,None,141.6,999.53,0.147,0.212,0.142,0,17830564000,9070422000,141.6,0.075,0.035,3224.72,8,14,6,0,0,19.72,19.49,2.875,13.38,3.521,14.93,0.719,3256.37,2277.88,3008.53,2736.24,17144000,None,None
4,BBWI,Common Stock,Bath & Body Works Inc.,None,701985,NYSE,USD,USA,TRADE & SERVICES,RETAIL-WOMEN'S CLOTHING STORES,"THREE LIMITED PKWY, P O BOX 16000, COLUMBUS, O...",January,2024-01-31,10892505000,1554000000,12.58,1.27,-7.23,0.8,0.0166,3.84,32.58,0.118,0.239,0.147,0,7429000000,3255000000,3.84,0.352,0.008,51.57,2,9,8,1,0,12.58,14.64,1.466,-,2.059,9.35,1.883,50.82,27.02,46.81,40.08,223665000,2024-06-21,2024-06-07
5,BBY,Common Stock,Best Buy Co. Inc,"Best Buy Co., Inc. is an American multinationa...",764478,NYSE,USD,USA,TRADE & SERVICES,"RETAIL-RADIO, TV & CONSUMER ELECTRONICS STORES","7601 PENN AVE SOUTH, RICHFIELD, MN, US",January,2024-01-31,15469168000,2650000000,12.59,1.439,14.17,3.68,0.0526,5.68,199.6,0.0286,0.0498,0.0702,0.424,43451998000,9912000000,5.68,-0.054,-0.006,83.93,2,6,18,1,1,12.59,12.89,0.387,5.5,0.445,7.45,1.516,85.08,60.79,76.36,73.78,216352000,2024-04-11,2024-03-20
6,BKNG,Common Stock,Booking Holdings Inc,Booking Holdings Inc. is an American travel te...,1075531,NASDAQ,USD,USA,ENERGY & TRANSPORTATION,TRANSPORTATION SERVICES,"800 CONNECTICUT AVE, NORWALK, CT, US",December,2024-03-31,128766738000,6808000000,28.57,1.009,-118.94,8.75,0.0092,132.84,623.53,0.218,0.184,0.148,2.257,22002000000,14625000000,132.84,2.196,0.169,3739.25,5,18,12,0,0,28.57,20.16,5.65,34.36,5.75,17.45,1.4,3908.0,2450.66,3600.43,3351.48,33927500,2024-06-28,2024-06-07
7,BWA,Common Stock,BorgWarner Inc,BorgWarner Inc. is an American multinational a...,908255,NYS

In [10]:
print(f'This is {pivot_sector_ticker_df.columns[2]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[2]]

This is Consumer Staples dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,MO,Common Stock,Altria Group,"Altria Group, Inc. (previously known as Philip...",764180,NYSE,USD,USA,MANUFACTURING,CIGARETTES,"6601 WEST BROAD STREET, RICHMOND, VA, US",December,2024-03-31,78134993000,12306000000,9.52,6.31,-2.977,3.88,0.0862,4.78,11.56,0.414,0.577,0.205,0,20455999000,14183000000,4.78,0.21,-0.01,43.04,2,3,6,3,0,9.52,8.17,3.439,27.3,4.539,7.54,0.682,46.6,37.33,43.58,42.18,1717630000,2024-07-10,2024-06-14
1,ADM,Common Stock,Archer-Daniels-Midland Company,"The Archer-Daniels-Midland Company, commonly k...",7084,NYSE,USD,USA,MANUFACTURING,FATS & OILS,"77 W. WACKER DR., CHICAGO, IL, US",December,2024-03-31,29908556000,4518000000,10.56,16.43,46.25,1.85,0.0331,5.73,172.47,0.0332,0.0326,0.0386,0.124,91709997000,7570000000,5.73,-0.33,-0.092,63.18,1,1,13,1,0,10.56,11.4,0.326,1.268,0.413,6.47,0.78,84.8,49.84,61.53,67.66,494438000,2021-09-08,2024-05-15
2,BG,Common Stock,Bunge Limited,Bunge Limited is a global food and agribusines...,1996862,NYSE,USD,USA,MANUFACTURING,FATS & OILS,"ROUTE DE FLORISSANT 13, GENEVA, CH",December,2024-03-31,14693314000,3403000000,8.37,1.705,74.52,2.65,0.0262,12.4,391.61,0.0322,0.0371,0.0695,0.173,57629000000,3918000000,12.4,-0.595,-0.125,115.61,4,5,4,0,0,8.37,11.36,0.246,1.348,0.297,4.4,0.648,113.58,84.91,103.46,102.7,141595000,2025-03-04,2024-08-19
3,CPB,Common Stock,Campbell Soup Company,"Campbell Soup Company, doing business as Campb...",16732,NYSE,USD,USA,MANUFACTURING,FOOD AND KINDRED PRODUCTS,"CAMPBELL PL, CAMDEN, NJ, US",July,2024-01-31,13605401000,1742000000,17.78,1.331,12.92,1.48,0.0326,2.55,31.06,0.0826,0.151,0.0706,0.206,9271000000,2935000000,2.55,-0.117,-0.012,46.13,1,3,13,3,2,17.78,13.04,1.361,3.277,1.83,10.55,0.218,50.38,37.31,44.5,42.73,298103000,2024-07-29,2024-07-03
4,CHD,Common Stock,Church & Dwight Company Inc,"Church & Dwight Co., Inc., is a major American...",313927,NYSE,USD,USA,LIFE SCIENCES,"SOAP, DETERGENTS, CLEANG PREPARATIONS, PERFUME...","500 CHARLES EWING BOULEVARD, EWING, NJ, US",December,2024-03-31,26034364000,1272500000,33.69,3.778,16.77,1.101,0.0107,3.16,24.27,0.131,0.203,0.0795,0.201,5941400000,2250000000,3.16,0.134,0.051,108.35,3,6,10,2,1,33.69,29.5,4.202,6.39,4.553,20.5,0.507,108.39,81.55,104.98,97.37,244523000,2024-06-03,2024-05-14
5,CLX,Common Stock,The Clorox Company,"The Clorox Company, based in Oakland, Californ...",21076,NYSE,USD,USA,LIFE SCIENCES,"SPECIALTY CLEANING, POLISHING AND SANITATION P...","THE CLOROX COMPANY, 1221 BROADWAY, OAKLAND, CA...",June,2024-03-31,16334448000,1143000000,68.15,0.433,0.733,4.8,0.0365,1.93,58.11,0.0333,0.129,0.0984,1.18,7209000000,2905000000,1.93,-0.064,-0.053,150.04,0,1,15,4,0,68.15,21.01,2.41,332.46,2.79,41.71,0.426,163.49,112.78,143.78,142.66,124188000,2024-05-10,2024-04-23
6,KO,Common Stock,The Coca-Cola Company,The Coca-Cola Company is an American multinati...,21344,NYSE,USD,USA,MANUFACTURING,BEVERAGES,"ONE COCA COLA PLAZA, ATLANTA, GA, US",December,2024-03-31,267093508000,14655000000,24.9,2.934,6.13,1.865,0.0313,2.49,10.67,0.234,0.33,0.0861,0.393,46073999000,25004000000,2.49,0.026,0.029,62.29,5,15,6,0,0,24.9,20.75,5.49,9.69,6.11,17.92,0.575,63.76,50.74,61.05,59.08,4307960000,2024-07-01,2024-06-14
7,CL,Common Stock,Colgate-Palmolive Company,Colgate-Palmolive Company is an American multi...,21665,NYSE,USD,USA,LIFE SCIENCES,"PERFUMES

In [11]:
print(f'This is {pivot_sector_ticker_df.columns[3]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[3]]

This is Energy dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,APA,Common Stock,APA Corporation,APA Corporation is the holding company for Apa...,6769,NASDAQ,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,"2000 POST OAK BLVD, STE 100, HOUSTON, TX, US",December,2024-03-31,10898198000,4942000000,3.295,0.17,8.66,1,0.0341,8.91,26.44,0.34,0.31,0.14,1.216,8083000000,7636000000,8.91,-0.438,-0.005,40.64,4,7,13,4,0,3.295,6.98,1.548,4.718,2.143,3.735,3.293,45.16,28.9,32.33,35.72,371192000,2024-05-22,2024-07-22
1,BKR,Common Stock,Baker Hughes Co,Baker Hughes Company is an American internatio...,1701605,NASDAQ,USD,USA,TECHNOLOGY,OIL & GAS FIELD MACHINERY & EQUIPMENT,"17021 ALDINE WESTFIELD ROAD, HOUSTON, TX, US",December,2024-03-31,32515987000,3895000000,18.08,0.96,15.43,0.8,0.0259,1.79,26.08,0.0695,0.103,0.0484,0.122,26208000000,4431000000,1.79,-0.211,0.123,40.82,6,16,5,0,0,18.08,16.45,1.302,2.147,1.426,9.19,1.434,36.85,26.29,32.9,33.19,997998000,2024-05-16,2024-05-03
2,CVX,Common Stock,Chevron Corp,Chevron Corporation is an American multination...,93410,NYSE,USD,USA,ENERGY & TRANSPORTATION,PETROLEUM REFINING,"6001 BOLLINGER CANYON ROAD, SAN RAMON, CA, US",December,2024-03-31,288493208000,41231000000,14.51,4.506,86.97,6.16,0.0413,10.87,103.61,0.105,0.139,0.0615,0.126,192786006000,98886000000,10.87,-0.142,-0.041,170.19,4,14,7,0,0,14.51,12.48,1.508,1.824,1.555,6.41,1.107,166.4,136.73,160.17,155.45,1828800000,2024-06-10,2024-05-16
3,COP,Common Stock,ConocoPhillips,ConocoPhillips is an American multinational co...,1163165,NYSE,USD,USA,ENERGY & TRANSPORTATION,PETROLEUM REFINING,"925 N. ELDRIDGE PARKWAY, HOUSTON, TX, US",December,2024-03-31,137595224000,24287001000,13.28,0.721,42.17,4.18,0.0277,8.83,47.66,0.186,0.265,0.105,0.218,56819999000,39598000000,8.83,-0.097,-0.068,143.64,7,15,6,0,0,13.28,13.97,2.655,3.118,2.957,6.48,1.264,134.54,95.77,125.77,118.44,1169530000,2024-06-03,2024-05-10
4,CTRA,Common Stock,Coterra Energy Inc,"Coterra Energy Inc., an independent oil and ga...",858470,NYSE,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,"840 GESSNER ROAD, SUITE 1400, HOUSTON, TX, US",December,2024-03-31,20191042000,3420000000,15.77,44.67,17.57,0.81,0.031,1.72,7.29,0.237,0.32,0.0522,0.101,5484000000,7636000000,1.72,-0.467,-0.127,33.5,4,12,11,0,0,15.77,15.27,3.653,1.593,3.777,5.79,0.205,29.22,22.23,27.78,26.83,744233000,2024-05-30,2024-05-15
5,DVN,Common Stock,Devon Energy Corporation,Devon Energy Corporation is an American energy...,1090012,NYSE,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,"333 W. SHERIDAN AVENUE, OKLAHOMA CITY, OK, US",December,2024-03-31,30430802000,7140000000,9.17,0.395,19.23,2.42,0.0426,5.25,22.69,0.232,0.237,0.115,0.289,14409000000,11329000000,5.25,-0.386,-0.005,58.37,4,16,10,0,0,9.17,10.27,2.351,2.812,2.589,5.23,2.186,55.09,40.09,50.82,47.22,632000000,2024-06-28,2024-06-14
6,FANG,Common Stock,Diamondback Energy Inc,Diamondback Energy is a company engaged in hyd...,1539838,NASDAQ,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,"500 WEST TEXAS, SUITE 1210, MIDLAND, TX, US",December,2024-03-31,34502431000,6344000000,10.91,1.201,95.04,9.26,0.0479,17.74,46.19,0.387,0.524,0.0989,0.199,8274000000,8173000000,17.74,0.102,0.174,224.32,4,18,4,0,0,10.91,11.52,4.623,2.213,5.16,6.97,1.895,209.88,116.96,199.97,168.85,178344000,2024-05-22,2024-05-14
7,EO

In [12]:
print(f'This is {pivot_sector_ticker_df.columns[4]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[4]]

This is Financials dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,Information
0,AFL,Common Stock,Aflac Incorporated,Aflac Inc. (American Family Life Assurance Com...,4977,NYSE,USD,USA,FINANCE,ACCIDENT & HEALTH INSURANCE,"1932 WYNNTON RD, COLUMBUS, GA, US",December,2024-03-31,49827389000,6321500000,9.65,0.93,41.27,1.76,0.023,9.09,32.94,0.277,0.408,0.0303,0.247,19337001000,8080000000,9.09,0.675,0.133,85.25,1,2,10,1,0,9.65,13.02,2.602,2.189,2.797,-,0.91,88.08,62.65,84.46,80.83,568222000,2024-06-03,2024-05-21,NaN
1,ALL,Common Stock,The Allstate Corporation,The Allstate Corporation is an American insura...,899051,NYSE,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","2775 SANDERS ROAD, NORTHBROOK, IL, US",December,2024-03-31,43503747000,2723000000,35.99,3.397,63.05,3.59,0.0224,4.58,223.11,0.0231,0.103,0.0126,0.0729,58567000000,6443000000,4.58,-0.676,0.107,189.63,6,11,4,1,0,35.99,12.85,0.771,2.79,0.932,-,0.514,177.37,98.51,168.71,142.49,263915000,2024-07-01,2024-05-31,NaN
2,AXP,Common Stock,American Express Company,The American Express Company is a multinationa...,4962,NYSE,USD,USA,FINANCE,FINANCE SERVICES,"200 VESEY STREET, 50TH FLOOR, NEW YORK, NY, US",December,2024-03-31,171323572000,None,19.6,2.247,40.01,2.5,0.0118,12.15,78.0,0.158,0.217,0.0356,0.329,56897999000,28776000000,12.15,0.388,0.099,214.4,5,11,9,2,2,19.6,16.86,2.826,5.6,-,-,1.203,244.41,140.02,230.32,188.79,719303000,2024-08-09,2024-07-05,NaN
3,AIG,Common Stock,American International Group Inc,"American International Group, Inc., also known...",5272,NYSE,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","175 WATER STREET, NEW YORK, NY, US",December,2024-03-31,51786011000,11047000000,11.66,0.549,64.66,1.44,0.0205,6.69,68.56,0.0998,0.183,0.0082,0.117,48367002000,24968000000,6.69,57,0.154,84.88,4,9,7,0,0,11.66,9.59,1.078,1.107,1.49,-,1.027,80.83,51.6,76.91,67.98,663668000,2024-06-28,2024-06-14,NaN
4,AMP,Common Stock,Ameriprise Financial Inc,"Ameriprise Financial, Inc. is a diversified fi...",820027,NYSE,USD,USA,FINANCE,INVESTMENT ADVICE,"1099 AMERIPRISE FINANCIAL CENTER, MINNEAPOLIS,...",December,2024-03-31,43799298000,4269000000,15.01,1.07,48.96,5.4,0.0134,29.37,158.74,0.189,0.368,0.0212,0.694,16576000000,7844000000,29.37,1.496,0.125,463.8,4,4,5,1,0,15.01,12.2,2.575,8.76,2.552,-,1.351,442.09,292.31,426.04,376.02,99325000,2024-05-17,2024-05-03,NaN
5,AON,Common Stock,Aon PLC,Aon plc is a multinational professional servic...,315293,NYSE,USD,USA,FINANCE,"INSURANCE AGENTS, BROKERS & SERVICE","THE LEADENHALL BUILDING, 122 LEADENHALL STREET...",December,2024-03-31,61043757000,4407000000,21.93,1.523,-0.67,2.46,0.0096,12.8,67.29,0.19,0.387,0.0692,1.761,13575000000,5823000000,12.8,0.055,0.051,320.73,1,3,11,3,0,21.93,19.05,4.566,89.18,5.38,18.41,0.892,344.47,267.42,304.7,314.71,217431000,2024-05-15,2024-04-30,NaN
6,ACGL,Common Stock,Arch Capital Group Ltd,"Arch Capital Group Ltd., offers insurance, rei...",947484,NASDAQ,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","WATERLOO HOUSE, GROUND FLOOR, 100 PITTS BAY RO...",December,2024-03-31,38811062000,4060000000,8.16,2.24,49.56,None,None,12.67,38.98,0.337,0.298,0.0436,0.29,14407000000,2847711000,12.67,0.562,0.244,108.21,4,9,4,0,0,8.16,10.91,2.476,1.927,2.752,-,0.566,103.79,69.05,94.46,84.58,375494000,None,None,NaN
7,AJG,Common Stock,Arthur J Gallagher & Co,Arthur J. Gallagher & Co. (AJG) is an American...

In [13]:
print(f'This is {pivot_sector_ticker_df.columns[5]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[5]]

This is Health Care dataframe over view


,Information
0,Thank you for using Alpha Vantage! Please cont...
1,Thank you for using Alpha Vantage! Please cont...
2,Thank you for using Alpha Vantage! Please cont...
3,Thank you for using Alpha Vantage! Please cont...
4,Thank you for using Alpha Vantage! Please cont...
5,Thank you for using Alpha Vantage! Please cont...
6,Thank you for using Alpha Vantage! Please cont...
7,Thank you for using Alpha Vantage! Please cont...
8,Thank you for using Alpha Vantage! Please cont...
9,Thank you for using Alpha Vantage! Please cont...


In [14]:
print(f'This is {pivot_sector_ticker_df.columns[6]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[6]]

This is Industrials dataframe over view


,Information,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,Thank you for using Alpha Vantage! Please cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thank you for using Alpha Vantage! Please cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thank you for using Alpha Vantage! Please cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thank you for using Alpha Vantage! Please cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thank you for using Alpha Vantage! Please cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,ADP,Common Stock,Automatic Data Processing Inc,"Automatic Data Processing, Inc. (ADP) is an Am...",8670,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-COMPUTER PROCESSING & DATA PREPARATION,"ONE ADP BOULVARD, ROSELAND, NJ, US",June,2024-03-31,101872525000,5487200000,27.81,2.671,11.29,5.3,0.0223,8.95,45.97,0.196,0.296,0.0496,0.889,18911900000,8510000000,8.95,0.147,0.066,261.24,1,2,15,2,0,27.81,24.1,5.47,23.0,5.72,18.35,0.785,255.1,201.57,245.91,241.86,409291000,2024-07-01,2024-06-14
6,NaN,AXON,Common Stock,Axon Enterprise Inc.,"Axon Enterprise, Inc. develops, manufactures, ...",1069183,NASDAQ,USD,USA,MANUFACTURING,"ORDNANCE & ACCESSORIES, (NO VEHICLES/GUIDED MI...","17800 N. 85TH ST., SCOTTSDALE, AZ, US",December,2024-03-31,21130060000,194274000,81.39,2.86,24.09,None,None,3.44,22.45,0.156,0.0353,0.0296,0.163,1681084000,728638000,3.44,1.836,0.343,344.71,4,8,2,0,0,81.39,68.03,15.0,14.55,14.61,117.35,0.907,329.87,175.37,306.54,252.9,75467000,None,None
7,NaN,BA,Common Stock,The Boeing Company,The Boeing Company is an American multinationa...,12927,NYSE,USD,USA,MANUFACTURING,AIRCRAFT,"P O BOX 3707 MS 1F 31, SEATTLE, WA, US",December,2024-03-31,107135042000,2675000000,None,6.53,-27.73,None,None,-3.54,125.63,-0.0281,0.0041,0.0038,0,76442001000,5768000000,-3.54,-0.68,-0.075,207.17,9,11,9,1,1,-,222.22,1.336,-,1.807,60.73,1.542,267.54,159.7,178.79,204.27,613884000,2020-03-06,2020-02-13
8,NaN,BR,Common Stock,Broadridge Financial Solutions Inc,Broadridge Financial Solutions is a public cor...,1383312,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1981 MARCUS AVENUE, LAKE SUCCESS, NY, US",June,2024-03-31,23699816000,1490900000,34.22,1.677,19.21,3.125,0.016,5.86,54.32,0.109,0.175,0.0773,0.328,6401500000,1785400000,5.86,0.072,0.049,208.79,0,4,5,0,0,34.22,23.15,3.705,11.32,4.274,20.3,0.987,209.41,142.11,199.34,192.41,118180000,2024-07-05,2024-06-14
9,NaN,BLDR,Common Stock,Builders FirstSource Inc,"Builders FirstSource, Inc., manufactures and s...",1316835,NYSE,USD,USA,TRADE & SER

In [15]:
print(f'This is {pivot_sector_ticker_df.columns[7]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[7]]

This is Information Technology dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",August,2024-02-29,192347046000,11277211000,27.31,2.194,43.13,4.99,0.0171,11.02,102.56,0.109,0.137,0.129,0.274,64573604000,20731607000,11.02,0.101,-0.001,383.36,7,10,10,0,0,27.31,26.11,3.071,7.31,3.041,18.21,1.207,386.02,286.3,323.43,332.29,628729000,2024-05-15,2024-04-10
1,ADBE,Common Stock,Adobe Systems Incorporated,Adobe Inc. is an American multinational comput...,796343,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"345 PARK AVE, SAN JOSE, CA, US",November,2024-02-29,212992639000,7589000000,45.45,1.759,34.13,None,None,10.46,43.76,0.241,0.368,0.157,0.324,19936000000,15441000000,10.46,-0.498,0.113,572.07,12,17,7,2,0,45.45,26.39,10.65,13.74,10.52,29.33,1.281,638.25,410.7,486.56,548.27,448000000,None,2005-03-24
2,AMD,Common Stock,Advanced Micro Devices Inc,"Advanced Micro Devices, Inc. (AMD) is an Ameri...",2488,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,"2485 AUGUSTINE DRIVE, SANTA CLARA, CA, US",December,2024-03-31,268889342000,3837000000,244.65,0.662,34.73,None,None,0.68,14.11,0.049,0.0066,0.0054,0.0201,22799999000,12051000000,0.68,48.69,0.022,172.75,14,25,11,0,0,244.65,48.78,11.64,4.721,11.51,62.93,1.66,227.3,93.11,164.47,141.31,1616310000,None,1995-04-27
3,AKAM,Common Stock,Akamai Technologies Inc,"Akamai Technologies, Inc. is a global content ...",1086222,NASDAQ,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","145 BROADWAY, CAMBRIDGE, MA, US",December,2024-03-31,14339121000,1110921000,23.48,1.278,30.4,None,None,4.01,25.63,0.161,0.17,0.0481,0.142,3883192000,2237817000,4.01,0.79,0.078,115.19,3,9,8,2,0,23.48,15.41,4.159,3.448,5.12,15.73,0.716,129.17,86.94,102.24,109.13,152317000,None,None
4,APH,Common Stock,Amphenol Corporation,Amphenol Corporation is a major producer of el...,820313,NYSE,USD,USA,MANUFACTURING,ELECTRONIC CONNECTORS,"358 HALL AVE, WALLINGFORD, CT, US",December,2024-03-31,81922343000,3095400000,41.66,5.85,14.45,0.86,0.0065,3.27,21.48,0.159,0.21,0.104,0.255,12837000000,4028200000,3.27,0.225,0.095,132.95,3,3,3,0,0,41.66,34.6,5.43,8.17,5.64,23.61,1.279,138.59,73.78,119.67,99.54,600604000,2024-07-10,2024-06-18
5,ADI,Common Stock,Analog Devices Inc,"Analog Devices, Inc. (ADI), also known simply ...",6281,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,"ONE TECHNOLOGY WAY, NORWOOD, MA, US",October,2024-04-30,115375407000,4868165000,54.32,3.614,71.03,3.56,0.0158,4.28,21.05,0.204,0.182,0.0342,0.0601,10464722000,7877218000,4.28,-0.683,-0.338,251.02,9,11,12,0,0,54.32,31.45,8.24,2.682,8.73,18.17,1.144,241.88,153.54,200.61,187.48,496217000,2024-03-15,2024-06-04
6,ANSS,Common Stock,ANSYS Inc,"Ansys, Inc. is an American company based in Ca...",1013462,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"2600 ANSYS DRIVE, SOUTHPOINTE, CANONSBURG, PA, US",December,2024-03-31,28905902000,677695000,66.35,2.027,61.97,None,None,4.99,25.64,0.195,0.0929,0.0497,0.0853,2227107000,1884284000,4.99,-0.653,-0.084,343.39,1,3,9,1,0,66.35,33.44,12.36,5.26,12.49,36.74,1.155,364.31,258.01,333.01,317.5,87300000,None,None
7,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno..

In [16]:
print(f'This is {pivot_sector_ticker_df.columns[8]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[8]]

This is Materials dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,APD,Common Stock,Air Products and Chemicals Inc,"Air Products and Chemicals, Inc. is an America...",2969,NYSE,USD,USA,LIFE SCIENCES,INDUSTRIAL INORGANIC CHEMICALS,"7201 HAMILTON BLVD, ALLENTOWN, PA, US",September,2024-03-31,58824389000,4115100000,23.93,1.372,67.08,7.02,0.0268,11.06,54.64,0.203,0.227,0.0518,0.161,12152800000,3767000000,11.06,0.305,-0.084,267.85,4,9,11,1,0,23.93,18.66,4.143,3.461,4.989,13.79,0.808,301.67,210.69,242.97,262.57,222306000,2024-08-12,2024-07-01
1,ALB,Common Stock,Albemarle Corp,Albemarle Corporation is a fine chemical manuf...,915913,NYSE,USD,USA,LIFE SCIENCES,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...","4250 CONGRESS STREET, SUITE 900, CHARLOTTE, NC...",December,2024-03-31,14995270000,-317096992,46.23,1.508,79.11,1.6,0.0125,2.76,71.55,0.0402,-0.106,-0.0268,0.0386,8397687000,3077787000,2.76,-0.662,-0.473,150.34,6,11,9,2,0,46.23,36.63,1.498,1.531,1.851,22.45,1.652,245.42,106.35,124.31,138.83,117527000,2024-07-01,2024-06-14
2,AMCR,Common Stock,Amcor PLC,"Amcor plc is an Australian-American, UK-domici...",1748790,NYSE,USD,USA,MANUFACTURING,MISCELLANEOUS MANUFACTURING INDUSTRIES,"83 TOWER ROAD NORTH, WARMLEY, BRISTOL, GB",June,2024-03-31,14366679000,1869000000,22.09,7.58,2.731,0.498,0.0503,0.45,9.55,0.0474,0.0991,0.0472,0.16,13778000000,2725000000,0.45,0.084,-0.07,9.83,0,1,5,3,0,22.09,13.19,0.918,3.249,1.426,11.46,0.848,10.38,8.12,9.51,9.35,1445340000,2024-06-11,2024-05-21
3,AVY,Common Stock,Avery Dennison Corp,Avery Dennison Corporation is a multinational ...,8818,NYSE,USD,USA,MANUFACTURING,CONVERTED PAPER & PAPERBOARD PRODS (NO CONTANE...,"207 GOODE AVENUE, GLENDALE, CA, US",December,2024-03-31,18289650000,1335000000,33.24,2.281,27.35,3.24,0.0155,6.83,104.85,0.0656,0.124,0.08,0.261,8450600000,2404200000,6.83,0.43,0.042,237.93,2,7,3,1,0,33.24,22.37,2.033,7.99,2.396,18.02,0.919,229.98,156.22,219.38,198.91,80553000,2024-06-20,2024-06-04
4,BALL,Common Stock,Ball Corporation,Ball Corporation supplies aluminum packaging p...,9389,NYSE,USD,USA,MANUFACTURING,METAL CANS,"10 LONGS PEAK DRIVE, BROOMFIELD, CO, US",December,2024-03-31,21378834000,2048000000,33.12,1.466,23.35,0.8,0.0116,2.08,44.2,0.303,0.0772,0.0425,0.119,13922000000,2583000000,2.08,19.77,-0.036,71.93,3,3,10,1,0,33.12,21.83,1.47,5.47,2.032,14.55,0.87,71.32,42.52,67.36,57.79,310378000,2024-06-17,2024-06-03
5,CE,Common Stock,Celanese Corporation,Celanese Corporation is a Fortune 500 global t...,1306830,NYSE,USD,USA,LIFE SCIENCES,"PLASTIC MATERIAL, SYNTH RESIN/RUBBER, CELLULOS...","222 W. LAS COLINAS BLVD., SUITE 900N, IRVING, ...",December,2024-03-31,16696461000,1821000000,8.38,4.42,64.75,2.8,0.0183,18.24,98.19,0.186,0.0827,0.0264,0.293,10698000000,2380000000,18.24,0.321,-0.085,166.9,1,5,12,3,1,8.38,13.32,1.591,2.455,2.715,11.24,1.363,171.38,101.11,159.94,141.48,109220000,2024-05-13,2024-04-26
6,CF,Common Stock,CF Industries Holdings Inc,"CF Industries Holdings, Inc. is a North Americ...",1324404,NYSE,USD,USA,LIFE SCIENCES,AGRICULTURAL CHEMICALS,"4 PARKWAY NORTH, SUITE 400, DEERFIELD, IL, US",December,2024-03-31,14304549000,2666000000,12.81,0.644,29.51,1.7,0.0258,6.05,31.86,0.19,0.211,0.0802,0.177,6089000000,5861000000,6.05,-0.639,-0.269,82.39,1,5,14,1,0,12.81,12.72,2.211,2.564,2.392,4.864,1.02,86.45,58.7,79.58,79.53,182782000,2024-05-31,2024-05-14
7,CTVA,

In [17]:
print(f'This is {pivot_sector_ticker_df.columns[9]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[9]]

This is Real Estate dataframe over view


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,Information
0,ARE,Common Stock,Alexandria Real Estate Equities Inc,"Alexandria Real Estate Equities, Inc. is an Am...",1035443,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,"26 NORTH EUCLID AVENUE, PASADENA, CA, US",December,2024-03-31,20706148000,1784144000,110.65,844.2,107.05,5.02,0.0429,1.07,17.26,0.0664,0.281,0.0129,0.0167,2954953000,1806454000,1.07,1.205,0.097,143.58,7,6,1,0,0,110.65,16.69,7.38,1.154,11.23,22.36,1.083,132.74,88.92,121.89,115.94,174883000,2024-04-15,2024-03-27,NaN
1,AMT,Common Stock,American Tower Corp,American Tower Corporation (also referred to a...,1053507,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,"116 HUNTINGTON AVE, BOSTON, MA, US",December,2024-03-31,87137575000,7003500000,41.97,1.396,8.67,6.51,0.0349,4.43,24.05,0.184,0.436,0.039,0.172,11211100000,7447300000,4.43,1.722,0.024,223.59,4,13,5,0,0,41.97,24.69,7.51,19.93,11.56,21.46,0.705,217.15,150.46,185.54,188.67,466975000,2024-04-26,2024-06-14,NaN
2,AVB,Common Stock,AvalonBay Communities Inc,"AvalonBay Communities, Inc. is a publicly trad...",915912,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,"4040 WILSON BOULEVARD, STE 1000, ARLINGTON, VA...",December,2024-03-31,27774614000,1744550000,28.98,5.03,82.5,6.65,0.0348,6.74,19.92,0.338,0.325,0.0282,0.0835,2825516000,1689582000,6.74,0.162,0.065,204.94,3,7,12,1,0,28.98,37.45,9.39,2.218,12.24,17.27,0.912,200.71,157.56,188.57,180.23,142186000,2024-07-15,2024-06-28,NaN
3,BXP,Common Stock,Boston Properties Inc,"Boston Properties, Inc. is a publicly traded r...",1037540,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,"800 BOYLSTON STREET, SUITE 1900, BOSTON, MA, US",December,2024-03-31,10451430000,1876588000,48.61,1.164,37.3,3.92,0.0661,1.22,20.93,0.0585,0.297,0.0258,0.0353,3283242000,1952127000,1.22,0.024,0.059,70.24,2,5,14,1,0,48.61,25.06,2.941,1.619,7.52,14.44,1.16,71.79,44.63,61.92,62.67,157059000,2024-04-30,2024-03-27,NaN
4,CPT,Common Stock,Camden Property Trust,"Camden Property Trust, an S&P 400 company, is ...",906345,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,"11 GREENWAY PLAZA, SUITE 2400, HOUSTON, TX, US",December,2024-03-31,11037026000,900570000,25.39,8.96,45.3,4.03,0.0398,4.08,14.3,0.287,0.198,0.022,0.0906,1553747000,905813000,4.08,0.998,0.02,105.71,4,3,12,3,0,25.39,58.82,6.76,2.099,9.02,12.4,0.834,110.49,81.09,100.57,97.57,106535000,2024-04-17,2024-03-27,NaN
5,CBRE,Common Stock,CBRE Group Inc Class A,"CBRE Group, Inc. is an American commercial rea...",1138118,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE,"400 SOUTH HOPE STREET, 25TH FLOOR, LOS ANGELES...",December,2024-03-31,26900795000,1937400000,27.48,1.178,27.38,None,None,3.19,105.68,0.0306,0.0286,0.037,0.118,32473000000,6622158000,3.19,0.108,0.071,105.57,4,0,6,0,0,27.48,21.93,0.859,3.319,0.97,18.11,1.439,98.65,64.63,90.67,84.35,306824000,None,2024-04-19,NaN
6,CSGP,Common Stock,CoStar Group Inc,"CoStar Group, Inc. provides online market info...",1057352,NASDAQ,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1331 L STREET, NW, WASHINGTON, DC, US",December,2024-03-31,35101114000,282500000,119.35,8.64,17.94,None,None,0.72,6.23,0.117,-0.0652,0.0119,0.0411,2527000000,1768391000,0.72,-0.921,0.123,104.17,5,7,3,0,0,119.35,227.27,

In [20]:
print(f'This is {pivot_sector_ticker_df.columns[10]} dataframe over view')
sector_overview_dict[pivot_sector_ticker_df.columns[10]]

This is Utilities dataframe over view


,Information
0,Thank you for using Alpha Vantage! Please cont...
1,Thank you for using Alpha Vantage! Please cont...
2,Thank you for using Alpha Vantage! Please cont...
3,Thank you for using Alpha Vantage! Please cont...
4,Thank you for using Alpha Vantage! Please cont...
5,Thank you for using Alpha Vantage! Please cont...
6,Thank you for using Alpha Vantage! Please cont...
7,Thank you for using Alpha Vantage! Please cont...
8,Thank you for using Alpha Vantage! Please cont...
9,Thank you for using Alpha Vantage! Please cont...
